# Разложение Река

In [3]:
from typing import Tuple
import torch
from torch import Tensor

def t_matrix(
    dim: int,
    target: Tuple[int, int],
    theta: Tensor,   # теперь принимаем Tensor (для градиентов)
    phi: Tensor,     # тоже Tensor
    dtype=torch.complex128,
    device='cpu'
) -> Tensor:
    """
    Матричная форма 2x2 произвольной унитарной матрицы, которая действует на выделенные 2 элемента вектора амплитуд (m, n).

    Блок параметризуется следующим образом:
        [[cos(theta),        -exp(-i*phi) * sin(theta)],
         [exp(i*phi) * sin(theta),  cos(theta)]]

    Параметры:
        dim: размер полной матрицы
        target: кортеж (m, n) — индексы амплитуд, на которые действует унитарное преобразование
        theta: угол вращения
        phi: фазовый угол
        dtype: тип данных
        device: устройство

   Вывод:
        Полная dim x dim унитарная матрица с заполненным блоком. 
    """
    m, n = target

    # Создаём единичную матрицу нужного размера
    T = torch.eye(dim, dtype=dtype, device=device)

    # Вычисляем элементы 2x2 блока с использованием torch (дифференцируемо!)
    cos_t = torch.cos(theta)
    sin_t = torch.sin(theta)
    exp_iphi = torch.exp(1j * phi)

    # Заполняем 2x2 подматрицу
    T[m, m] = cos_t
    T[m, n] = -torch.conj(exp_iphi) * sin_t  # = -exp(-i*phi) * sin(theta)
    T[n, m] = exp_iphi * sin_t               # =  exp(i*phi) * sin(theta)
    T[n, n] = cos_t

    return T

def reck_decomposition(U: Tensor):
    """
    Декомпозиция Река на 2x2 блочные матрицы и диагональную матрицу фаз [1].
    
    Ввод:
        U: Матрица, которая будем расслад

    Вывод:
        thetas: Значения углов theta в каждом блоке
        phis:   Значения фаз phi в каждом блоке
        targets: Пары мод, между которыми ставиться блок
        phases: Диагональные фазосменщики
    
    [1] Experimental realization of any discrete unitary operator (1994), Reck et al.
    """
    N = U.shape[0]
    device = U.device
    dtype_real = torch.float64
    dtype_int = torch.int64

    # Инициализация списков (лучше, чем cat в цикле)
    thetas_list = []
    phis_list = []
    targets_list = []

    U_work = U.clone()

    for n in range(N - 1, 0, -1):          # столбцы: справа налево
        for m in range(n - 1, -1, -1):     # строки: снизу вверх в столбце n
            a = U_work[n, n]
            b = U_work[m, n]

            r = torch.sqrt(torch.abs(a)**2 + torch.abs(b)**2 + 1e-15)

            if r < 1e-12:
                theta = torch.tensor(0.0, dtype=dtype_real, device=device)
                phi = torch.tensor(0.0, dtype=dtype_real, device=device)
            else:
                cos_theta = torch.abs(a) / r
                cos_theta = torch.clamp(cos_theta, -1.0, 1.0)
                theta = torch.acos(cos_theta)
                phi = torch.angle(a) - torch.angle(b)

            thetas_list.append(theta)
            phis_list.append(phi)
            targets_list.append(torch.tensor([m, n], dtype=dtype_int, device=device))
            
            T_mn = t_matrix(N, target=(m,n), theta=theta, phi=phi)

            U_work = T_mn @ U_work

    thetas = torch.stack(thetas_list) if thetas_list else torch.empty(0, dtype=dtype_real, device=device)
    phis = torch.stack(phis_list) if phis_list else torch.empty(0, dtype=dtype_real, device=device)
    targets = torch.stack(targets_list) if targets_list else torch.empty((0, 2), dtype=dtype_int, device=device)

    phases = torch.angle(torch.diag(U_work))  # вещественные фазы

    return thetas, phis, targets, phases

def get_matrix(
    dim: int, 
    phases: Tensor,
    phis: Tensor,
    thetas: Tensor,
    targets: Tensor,
    dtype=torch.complex128,
    device='cpu'
) -> Tensor:
    """
    Получение унитарной матрицы из известных параметров после декомпозиции Река. 

    Ввод:
        dim: размер матрицы
        phases: Диагональные фазосменщики
        phis:   Значения фаз phi в каждом блоке
        thetas: Значения углов theta в каждом блоке
        targets: Пары мод, между которыми ставиться блок
        dtype: тип данных
        device: устройство
    
    Вывод:
        U: Унитарная матрица dim x dim, полученная в результате декомпозиции Река
        layers: Словарь номер_слоя -> тензор блочной матрицы
    """
    U = torch.diag(torch.exp(1j * phases.to(dtype))).to(device)
    layers = dict.fromkeys([i for i in range(dim*(dim - 1)//2 + 1)])
    layers[0] = U.clone()
    for i in range(len(thetas) - 1, -1, -1):
        theta = thetas[i]
        phi = phis[i]
        m, n = targets[i].tolist()
        T_mn = t_matrix(dim, target=(m,n), theta=theta, phi=phi)
        U = T_mn.adjoint() @ U
        layers[i+1] = T_mn.adjoint()
    return U, layers

def fidellity(U_exp: Tensor, U: Tensor):
    """
    Мера близости между двумя унитарными матрицами

    Fid = |Tr(U_{exp}^{dagger} U)|/N , где N - размерность матриц
    """
    return torch.abs(torch.einsum('ii', U_exp.adjoint() @ U)).item()/U.shape[0]

def haar_measure(N):
    """
    Генератор Хааровских матриц с помощью QR декомпозициия
    """
    A, B = torch.randn(size=(N, N), dtype=torch.float64), torch.randn(size=(N, N), dtype=torch.float64)
    Z = A + 1j * B
    Q, R = torch.linalg.qr(Z)
    Lambda = torch.diag(torch.tensor([R[i, i] / torch.abs(R[i, i]) for i in range(N)]))

    return Q @ Lambda

if __name__ == '__main__':
    torch.manual_seed(42)
    N = torch.randint(low=2, high=10, size = ())
    U = haar_measure(N)
    print(f"Случайная унитарная матрица размера {N} из распределения Хаара :\n {U.numpy().round(4)}")

    #Проводим разложение Река -> получаем параметры
    thetas, phis, targets, phases = reck_decomposition(U)

    #Восстанавливаем матрицу -> получаем матрицу + словарь с номером слоя и его матрицей
    reconstructed_unitary, layers = get_matrix(N, phases, phis, thetas, targets)

    print("\nВосстановленая с помощью декомпозиции Река унитарная матрица:\n", reconstructed_unitary.detach().numpy().round(4))
    print("\nФиделити:\n", fidellity(reconstructed_unitary, U))
    assert torch.allclose(U, reconstructed_unitary, atol=1e-06), "Восстановленая матрица не совпадает с входной"

    print("\nМатрицы для каждого слоя\n")
    for i in layers.keys():
        theta = thetas[i-1]
        phi = phis[i-1]
        m, n = targets[i-1].tolist()
        print(f'Номер слоя {i}: \t\t Параметры: theta: {theta:.3f}, phi: {phi:.2f}, target: {(m, n)} \n {layers[i]}')

Случайная унитарная матрица размера 8 из распределения Хаара :
 [[-0.2006+0.0945j  0.5269-0.099j  -0.2327+0.2757j  0.008 +0.1147j
  -0.0506+0.092j  -0.1041-0.2304j -0.3972+0.4805j -0.0954-0.2177j]
 [-0.1295-0.1587j  0.1815+0.1986j -0.0996-0.1983j -0.3697+0.4263j
  -0.1729-0.019j  -0.4346+0.3077j  0.2055+0.1765j  0.1376+0.3344j]
 [-0.3724-0.5003j -0.1125-0.2725j -0.0402+0.2994j -0.3849-0.0267j
   0.393 +0.2562j  0.1423+0.1155j -0.0635-0.1336j -0.0753+0.0523j]
 [-0.1528+0.2489j -0.0655+0.3764j  0.1486+0.1059j -0.2019-0.1234j
  -0.3107+0.3058j  0.3503+0.3919j  0.0303+0.1637j -0.4256-0.0662j]
 [-0.0238-0.1727j -0.3071+0.078j   0.1853-0.2987j  0.0207+0.0415j
   0.0188+0.4263j  0.1027-0.1532j  0.0292+0.4863j  0.4608-0.2788j]
 [ 0.1624+0.2351j  0.2311-0.0187j  0.1415-0.227j  -0.1423-0.3131j
   0.5125-0.1013j -0.0614+0.5134j -0.2805+0.1268j  0.1911-0.0567j]
 [-0.0476+0.3983j -0.4043-0.1464j -0.6899-0.1172j -0.0803-0.118j
   0.0564+0.1781j -0.0128-0.0217j -0.0923+0.1006j  0.0269+0.2987j]
 [ 0.3